# Rt2_assignment2

# Code

In [1]:
import rospy
import time
import jupyros as jr
import matplotlib
import matplotlib.pyplot as plt
import numpy as np 
import math
import actionlib
import actionlib.msg
import ipywidgets as widgets
from rt2_assignment1.srv import Command, SetVel
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry


%matplotlib widget
from matplotlib.animation import FuncAnimation
from matplotlib import animation, rc
from std_msgs.msg import String, Float64, Bool, Float32
from IPython.display import display
from ipywidgets import Button, GridBox, Layout, ButtonStyle, FloatSlider, HBox, VBox
from tf import transformations


## Classes

### ListLim

In [2]:
class ListLim(list):
    def __init__(self, max_size, l=[]):
        super().__init__(l)
        self.max_size = max_size
    def append(self, data):
        super().append(data)
        if super().__len__() > self.max_size:
            super().pop(0)

### Direction Pad

In [4]:
class DirectionPad:
    def __init__(self, lin_slider, ang_slider):
        self.vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
        self.twist = Twist()
        self.go_on_mov = 0.0
        self.ang_mov = 0.0
        self.lin_slider = lin_slider
        self.ang_slider = ang_slider
    
    def goForward(self):
        self.go_on_mov = 1.0
        self.ang_mov = 0.0
    def goBackward(self):
        self.go_on_mov = -1.0
        self.ang_mov = 0.0
    def turnRight(self):
        self.go_on_mov = 0.0
        self.ang_mov = 1.0
    def turnLeft(self):
        self.go_on_mov = 0.0
        self.ang_mov = -1.0
    def stopping(self):
        self.go_on_mov = 0.0
        self.ang_mov = 0.0
    
    def pubTwist(self):
        self.twist.linear.x = self.go_on_mov*lin_slider.value
        self.twist.linear.z = self.ang_mov*ang_slider.value
        
        self.vel_pub.publish(self.twist)

## Global variables

In [5]:
output = widgets.Output()

lin_init = 0.6
ang_init = 0.6

rob_odom_max_leng = 2000
rob_odom_x = ListLim(rob_odom_max_leng)
rob_odom_y = ListLim(rob_odom_max_leng)
rob_odom_rot = 0

vel_max_len = 200
vel_x_sim = ListLim(vel_max_len, [lin_init]*vel_max_len)
vel_x_odom = ListLim(vel_max_len, [0]*vel_max_len)
vel_w_sim = ListLim(vel_max_len, [ang_init]*vel_max_len)
vel_w_odom = ListLim(vel_max_len, [0]*vel_max_len)

t = np.linspace(0,1,vel_max_len)

from enum import Enum

class Running(Enum):
    STOP = 0
    GO = 1
    PAD = 2
running = Running.STOP

vel_client = None
ui_client = None

## Widgets

### Sliders

In [6]:
max_vel = 1.6
lin_slider = FloatSlider(description='Linear Velocity', value=lin_init, min=0.1, max=max_vel, step=0.02)
ang_slider = FloatSlider(description='Angular Velocity', value=ang_init, min=0.1, max=max_vel, step=0.02)

v1=VBox([lin_slider, ang_slider])
def lin_vel_change(vel):
    vel_client(vel["new"], vel_w_sim[-1])

def ang_vel_change(vel):
    vel_client(vel_x_sim[-1], vel["new"])
    
lin_slider.observe(lin_vel_change, names='value')
ang_slider.observe(ang_vel_change, names='value')

### Buttons

In [7]:
up_Button = Button(description='\u2191', layout=Layout(width='auto',grid_area='up'),
                  style=ButtonStyle(button_color='lightblue'))
down_Button = Button(description='\u2193', layout=Layout(width='auto',grid_area='down'),
                  style=ButtonStyle(button_color='lightblue'))
right_Button = Button(description='\u2192', layout=Layout(width='auto',grid_area='right'),
                  style=ButtonStyle(button_color='lightblue'))
left_Button = Button(description='\u2190', layout=Layout(width='auto',grid_area='left'),
                  style=ButtonStyle(button_color='lightblue'))

gb = GridBox(children=[up_Button, down_Button, right_Button, left_Button],
            layout=Layout(
                width='100%',
                grid_template_row='auto auto',
                grid_template_columns='33% 33% 33%',
                grid_templates_areas='''
                ".up"
                "left down right"
                ''')
            )

In [8]:
start_Button = Button(description='START', layout=Layout(width='100%',grid_area='start'),
                  style=ButtonStyle(button_color='lightblue'))
stop_Button = Button(description='START', layout=Layout(width='100%',grid_area='start'),
                  style=ButtonStyle(button_color='lightblue'))
h1=HBox([start_Button,stop_Button])

g_all = GridBox(children=[v1, h1, gb],
            layout=Layout(
                width='100%',
                grid_template_row='auto auto',
                grid_template_columns='33% 40% 27%',
                grid_templates_areas='''
                "v1 . gb"
                " . h1 ."
                ''')
            )